# CanOE Nitrate

Output from recent CanOE run provided by Hakase and Nadja. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
from salishsea_tools import nc_tools, viz_tools
import datetime
from mpl_toolkits.basemap import Basemap, cm
import cmocean

/ocean/brogalla/GEOTRACES/analysis-birgit/cmocean-master/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


Load ANHA12 coordinate file:

In [2]:
mesh = '/data/brogalla/ANHA12/ANHA12-EXH006_5d_gridT_y2002m01d05.nc'
m = nc.Dataset(mesh)
mdepth = m.variables['deptht']
mlons = m.variables['nav_lon']
mlats = m.variables['nav_lat']
mdepth = np.array(mdepth)
mlons = np.array(mlons)
mlats = np.array(mlats)
# Mask:
mask = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12_mask.nc')
tmask = mask.variables['tmask']
tmask = np.array(tmask[0,:,:,:])
cond = (tmask > 0.1) 
Z_masked = np.ma.masked_where(cond, tmask) 

In [4]:
b = load_data(2002,1)

In [5]:
print(np.amax(b))
print(np.amin(b))
print(np.average(b))

60.688379026
0.0
1.36341745755


### Functions:

In [3]:
def load_data(year, month):
    folder = '/ocean/brogalla/GEOTRACES/analysis-birgit/notebooks/bio/'
    filename = 'NO3_y'+str(year)+'m'+str(month)+'.nc'
    
    data  = nc.Dataset(folder+filename)
    NO3   = np.array(data.variables['NO3'])
    return NO3

In [35]:
def deriv(year, month):
    NO3_1 = load_data(year, month)
    if month < 12:
        NO3_2 = load_data(year, month+1)
    else:
        NO3_2 = load_data(year+1, 1)
        
    diff = np.subtract(NO3_2, NO3_1) / (30*24*60*60)
    return diff

In [18]:
def save_NO3(filename, field):
    c = nc.Dataset('/data/brogalla/ANHA12/ANHA12-EXH006_5d_gridT_y2002m01d05.nc','r')

    # Load structure from sample dataset:
    deptht = c.variables['deptht']
    BX = c.dimensions['x']
    BY = c.dimensions['y']
    
    ncd = nc.Dataset(filename, 'w', zlib=True)
    nc_tools.init_dataset_attrs(
        ncd,
        title='NO3 input field',
        notebook_name='IC',
        nc_filepath='./NO3_input.nc',
        comment='CanOE NO3 deposition field')

    ncd.createDimension('x',len(BX))
    ncd.createDimension('y',len(BY))
    ncd.createDimension('deptht', len(deptht))
    
    # variables
    NO3_var = ncd.createVariable('dNO3', 'float64', ('deptht','y','x'))
    NO3_var.units = ''
    NO3_var.long_name = 'Nitrate concentration'  
    NO3_var.coordinates = 'nav_lon nav_lat deptht'
    NO3_var[:] = field[:,:,:]
    
    nc_tools.check_dataset_attrs(ncd)
    ncd.close()
    return

In [6]:
def plot_sample(NO3, mlons=mlons, mlats=mlats):
    fig,ax = plt.subplots(figsize=(15,15))
    proj=Basemap(projection='eqdc',lat_0 = 77.5, lon_0 = -93, resolution='i', width=3.5e6, height=2.68e6)
    proj.drawcoastlines(linestyle='-', color=[0.35, 0.35, 0.35], linewidth=1, zorder=1);

    # model:
    x, y = proj(mlons, mlats)

    # plot all together:
    CS = proj.pcolor(x, y, NO3[0,0,:,:], cmap=cmocean.cm.deep)
    CBar = plt.colorbar(CS, shrink=0.6)
    CBar.ax.tick_params(axis='y', length=0)
    CBar.set_label('Interpolated NO3', fontsize=12)
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.title('ANHA12 interpolation from CanOE model output')
    return

## Run:

##### Choose year:

In [7]:
year = 2002 

Calculate derivative:

Final result has to be in units of [NO_3]/s

\begin{equation*}
\frac{\partial{NO_{3}(i)}}{\partial{t}} \approxeq \frac{NO_{3}(i+1) - NO_{3}(i) }{\Delta t} = 
\frac{NO_{3}(2002m2)-NO_{3}(2002m1)}{(30*24*60*60)} 
\end{equation*}

In [36]:
def joblib_solver(deriv, year, month):
    dNO3 = deriv(year, month) 
    return dNO3

In [37]:
months=np.arange(1,13)
joblist=[]
for month in months:
    positional_args=[deriv,year,month]
    keyword_args={}
    joblist.append((joblib_solver,positional_args,keyword_args))

In [38]:
from joblib import Parallel
ncores=8
with Parallel(n_jobs=ncores,backend='threading') as parallel:
    results=parallel(joblist)

In [39]:
a = np.array(results)
a.shape

(12, 50, 2400, 1632)

Check that there are no NaN values:

In [40]:
print(np.count_nonzero(np.isnan(a)))

0


In [41]:
print(np.amax(a))
print(np.average(a))
print(np.amin(a))

8.31980946882e-06
-1.45165734169e-10
-1.19999793556e-05


save to NetCDF file: collapse shape from (t,z,y,x) to (t,z,xy)

In [ ]:
time_dim=1
bound_dimE=len(boundaryE_lats)

In [ ]:
# final dimensions should be
rimwidth = 20
len(boundaryE_lats)*rimwidth

In [ ]:
# function to stack input for boundaries:
def stack_input(var, time_dim, bound_dim, rimwidth):
    a = var
    b = np.hstack((var,var))
    for i in range(0,rimwidth-2):
        b = np.hstack((b,a))
#     c = np.dstack((b, b))
#     for i in range(0,time_dim-2):
#         c = np.dstack((c, b))
    d = np.moveaxis(b,-1,0)
    return d

In [ ]:
# Eastern boundary:
dMnE_O = stack_input(boundaryE_dMn, time_dim, bound_dimE, rimwidth)
oMnE_O = stack_input(boundaryE_oMn, time_dim, bound_dimE, rimwidth)
print(oMnE_O.shape)
print(dMnE_O.shape)

In [ ]:
def flatten_input(var):
    b = var[0,:,:].flatten()
    for i in range(1,len(var)):
        a = var[i,:,:].flatten()
        b = np.vstack((b,a))
    return b

In [ ]:
dMnE_O = flatten_input(boundaryE_dMn)
oMnE_O = flatten_input(boundaryE_oMn)
print(dMnE_O.shape)
print(oMnE_O.shape)

In [ ]:
dMn_EOBC = np.reshape(dMnE_O, (1,50,1,bound_dimE*rimwidth))
oMn_EOBC = np.reshape(oMnE_O, (1,50,1,bound_dimE*rimwidth))
print(dMn_EOBC.shape)
print(oMn_EOBC.shape)

In [43]:
for i in range(1,13):
    if i < 10:
        save_NO3('dNO3_y'+str(year)+'m0'+str(i)+'.nc',a[i-1,:,:,:]) 
    else:
        save_NO3('dNO3_y'+str(year)+'m'+str(i)+'.nc',a[i-1,:,:,:])

file format: NETCDF4
Conventions: CF-1.6
title: NO3 input field
institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
source: https://bitbucket.org/ccar-modeling/analysis-birgit/src/tip/notebooks/bio/IC.ipynb
references: REQUIRED
history: [2018-08-17 10:09:33] Created netCDF4 zlib=True dataset.
comment: CanOE NO3 deposition field
1.43318257348e-08
Missing value for dataset attribute: references
Missing value for variable attribute for dNO3: units
file format: NETCDF4
Conventions: CF-1.6
title: NO3 input field
institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
source: https://bitbucket.org/ccar-modeling/analysis-birgit/src/tip/notebooks/bio/IC.ipynb
references: REQUIRED
history: [2018-08-17 10:09:50] Created netCDF4 zlib=True dataset.
comment: CanOE NO3 deposition field
6.58032858745e-09
Missing value for dataset attribute: references
Missing value for variable attribute for dNO3: units
file format: NETCDF4
Convent

In [44]:
m = nc.Dataset('./dNO3_y2002m01.nc')
m1 = m.variables['dNO3']
print(np.average(m1))

1.43318257348e-08


###  In case you want to look at all input fields:

In [ ]:
NO3 = i_NO3

fig = plt.figure(figsize=(12,12))
ax1 = plt.subplot2grid((4, 3), (0, 0)) 
ax2 = plt.subplot2grid((4, 3), (0, 1)) 
ax3 = plt.subplot2grid((4, 3), (0, 2)) 
ax4 = plt.subplot2grid((4, 3), (1, 0)) 
ax5 = plt.subplot2grid((4, 3), (1, 1)) 
ax6 = plt.subplot2grid((4, 3), (1, 2)) 
ax7 = plt.subplot2grid((4, 3), (2, 0)) 
ax8 = plt.subplot2grid((4, 3), (2, 1)) 
ax9 = plt.subplot2grid((4, 3), (2, 2)) 
ax10 = plt.subplot2grid((4, 3), (3, 0)) 
ax11 = plt.subplot2grid((4, 3), (3, 1)) 
ax12 = plt.subplot2grid((4, 3), (3, 2)) 

m=Basemap(projection='eqdc',lat_0 = 77.5, lon_0 = -93, resolution='i', width=3.5e6, height=2.68e6)
x1, y1 = m(lon,lat)
x2, y2 = m(mlons, mlats)
x2 = x2[::4,::4]
y2 = y2[::4,::4]
z2 = Z_masked[0,::4,::4]

m.pcolor(x1,y1,NO3[0,0,:,:], cmap=cmocean.cm.deep, vmin=0, vmax=75, ax=ax1)
ax1.set_title('Month 1')
m.pcolor(x2,y2,z2, vmin=-1, vmax = 0.3, cmap=cmocean.cm.matter_r, ax=ax1)    
m.pcolor(x1,y1,NO3[1,0,:,:], cmap=cmocean.cm.deep, vmin=0, vmax=75, ax=ax2)
m.pcolor(x2,y2,z2, vmin=-1, vmax = 0.3, cmap=cmocean.cm.matter_r, ax=ax2)    
ax2.set_title('Month 2')
d=m.pcolor(x1,y1,NO3[2,0,:,:], cmap=cmocean.cm.deep, vmin=0, vmax=75, ax=ax3)
m.pcolor(x2,y2,z2, vmin=-1, vmax = 0.3, cmap=cmocean.cm.matter_r, ax=ax3) 
ax3.set_title('Month 3')
cbaxes = fig.add_axes([0.975, 0.15, 0.02, 0.7]) 
CBar = plt.colorbar(d, ax=ax3, cax=cbaxes) 
a = CBar.set_label('NO3', fontsize=16)
CBar.ax.tick_params(axis='y', length=0, labelsize=16)

m.pcolor(x1,y1,NO3[3,0,:,:], cmap=cmocean.cm.deep, vmin=0, vmax=75, ax=ax4)
m.pcolor(x2,y2,z2, vmin=-1, vmax = 0.3, cmap=cmocean.cm.matter_r, ax=ax4) 
ax4.set_title('Month 4')
m.pcolor(x1,y1,NO3[4,0,:,:], cmap=cmocean.cm.deep, vmin=0, vmax=75, ax=ax5)
m.pcolor(x2,y2,z2, vmin=-1, vmax = 0.3, cmap=cmocean.cm.matter_r, ax=ax5) 
ax5.set_title('Month 5')
m.pcolor(x1,y1,NO3[5,0,:,:], cmap=cmocean.cm.deep, vmin=0, vmax=75, ax=ax6)
m.pcolor(x2,y2,z2, vmin=-1, vmax = 0.3, cmap=cmocean.cm.matter_r, ax=ax6) 
ax6.set_title('Month 6')
m.pcolor(x1,y1,NO3[6,0,:,:], cmap=cmocean.cm.deep, vmin=0, vmax=75, ax=ax7)
m.pcolor(x2,y2,z2, vmin=-1, vmax = 0.3, cmap=cmocean.cm.matter_r, ax=ax7) 
ax7.set_title('Month 7')
m.pcolor(x1,y1,NO3[7,0,:,:], cmap=cmocean.cm.deep, vmin=0, vmax=75, ax=ax8)
m.pcolor(x2,y2,z2, vmin=-1, vmax = 0.3, cmap=cmocean.cm.matter_r, ax=ax8) 
ax8.set_title('Month 8')
m.pcolor(x1,y1,NO3[8,0,:,:], cmap=cmocean.cm.deep, vmin=0, vmax=75, ax=ax9)
m.pcolor(x2,y2,z2, vmin=-1, vmax = 0.3, cmap=cmocean.cm.matter_r, ax=ax9) 
ax9.set_title('Month 9')
m.pcolor(x1,y1,NO3[9,0,:,:], cmap=cmocean.cm.deep, vmin=0, vmax=75, ax=ax10)
m.pcolor(x2,y2,z2, vmin=-1, vmax = 0.3, cmap=cmocean.cm.matter_r, ax=ax10) 
ax10.set_title('Month 10')
m.pcolor(x1,y1,NO3[10,0,:,:], cmap=cmocean.cm.deep, vmin=0, vmax=75, ax=ax11)
m.pcolor(x2,y2,z2, vmin=-1, vmax = 0.3, cmap=cmocean.cm.matter_r, ax=ax11) 
ax11.set_title('Month 11')
m.pcolor(x1,y1,NO3[11,0,:,:], cmap=cmocean.cm.deep, vmin=0, vmax=75, ax=ax12)
m.pcolor(x2,y2,z2, vmin=-1, vmax = 0.3, cmap=cmocean.cm.matter_r, ax=ax12) 
ax12.set_title('Month 12')